In [1]:
import os
import pathlib
import json

In [2]:
import pandas as pd 
import numpy as np

### enter bulletin directory 

In [3]:
bulletin_dir = 'JJA_2019'

In [4]:
dpath = pathlib.Path.cwd().parents[1] / 'bulletin' / bulletin_dir

In [5]:
dpath

PosixPath('/home/nicolasf/operational/ICU/ops/bulletin/JJA_2019')

### reads the country dictionnary for the TRMM rainfall time-series

In [6]:
with open(pathlib.Path.cwd().parents[1] / 'resources' / 'dict_countries.json', 'r') as fj: 
    dict_countries = json.load(fj)

### read the forecast table 

In [7]:
forecast = pd.read_csv(dpath / 'copernicus_forecasts_{}.csv'.format(bulletin_dir), index_col=0)

In [8]:
forecast.head()

,T1,T2,T3,country_fname,std,CONFIDENCE
country_name,,,,,,
Nauru,2,2,96,Nauru,26.095783,High
Kiribati: Gilbert Islands,3,4,93,Kiribati:_Gilbert_Islands,19.975656,High
Federated States of Micronesia,16,20,64,Federated_States_of_Micronesia,11.410921,High
Kiribati: Phoenix Islands,18,23,59,Kiribati:_Phoenix_Islands,15.900944,Moderate-High
Tuvalu,21,25,54,Tuvalu,20.624085,Moderate


In [9]:
def terciles_to_category(df, colnames=['T1','T2','T3'], mindiff=10, climdiff=5): 
    """
    convert a tercile probabilistic forecast into a categorical forecast
    
    if the difference between the maximum probability and the next one is >= mindiff 
    (10 is default) then it's either 'ABOVE', 'NEAR NORMAL' or 'BELOW' depending on the max 
    probability 
    
    if the difference between consecutive probabilities is less than mindiff (10)
    but greater than climdiff (5) then we end up with either 'AVG - BELOW' or 'AVG - ABOVE'
    
    if the difference between consecutive probabilities is less than 5%, it's a CLIMATOLOGY 
    forecasts 
    """
    
    T1, T2, T3 = colnames
    
    cats = []
    for i, row in df.iterrows(): 
        fcst = row.loc[colnames,].astype('int')
        if (fcst.idxmax()  == T3) & ((fcst.loc[T3] - fcst.loc[T2]) >= mindiff): 
            cat = 'ABOVE'
#         elif (fcst.idxmax()  == T3) & ((fcst.loc[T3] - fcst.loc[T2]) < mindiff) & ((fcst.loc[T3] - fcst.loc[T2]) > climdiff) & ((fcst.loc[T3] - fcst.loc[T1]) > mindiff):
        elif (fcst.idxmax()  == T3) & ((fcst.loc[T3] - fcst.loc[T2]) < mindiff) & ((fcst.loc[T3] - fcst.loc[T1]) > mindiff):
            cat = 'AVG - ABOVE'
        elif (fcst.idxmax()  == T1) & ((fcst.loc[T1] - fcst.loc[T2]) >= mindiff):
            cat = 'BELOW'
#         elif (fcst.idxmax()  == T1) & ((fcst.loc[T1] - fcst.loc[T2]) < mindiff) & ((fcst.loc[T1] - fcst.loc[T2]) > climdiff) & ((fcst.loc[T1] - fcst.loc[T3]) > mindiff):
        elif (fcst.idxmax()  == T1) & ((fcst.loc[T1] - fcst.loc[T2]) < mindiff) & ((fcst.loc[T1] - fcst.loc[T3]) > mindiff):
            cat = 'AVG - BELOW'
        elif (fcst.idxmax()  == T2) & ((fcst.loc[T2] - fcst.loc[T1]) >= mindiff) & ((fcst.loc[T2] - fcst.loc[T3]) >= mindiff): 
            cat = 'NEAR NORMAL'
        else: 
            cat = 'CLIMATOLOGY'
        cats.append(cat)
    return np.array(cats)

In [10]:
forecast

,T1,T2,T3,country_fname,std,CONFIDENCE
country_name,,,,,,
Nauru,2,2,96,Nauru,26.095783,High
Kiribati: Gilbert Islands,3,4,93,Kiribati:_Gilbert_Islands,19.975656,High
Federated States of Micronesia,16,20,64,Federated_States_of_Micronesia,11.410921,High
Kiribati: Phoenix Islands,18,23,59,Kiribati:_Phoenix_Islands,15.900944,Moderate-High
Tuvalu,21,25,54,Tuvalu,20.624085,Moderate
Kiribati: Line Islands,25,32,43,Kiribati:_Line_Islands,14.624954,High
Austral Islands,26,32,42,Austral_Islands,11.493066,High
Tokelau,30,30,40,Tokelau,20.133254,Moderate
Pitcairn Islands,29,32,39,Pitcairn_Islands,13.405402,High


In [11]:
forecast.loc[:,'CATEGORY'] = terciles_to_category(forecast)

In [12]:
forecast

,T1,T2,T3,country_fname,std,CONFIDENCE,CATEGORY
country_name,,,,,,,
Nauru,2,2,96,Nauru,26.095783,High,ABOVE
Kiribati: Gilbert Islands,3,4,93,Kiribati:_Gilbert_Islands,19.975656,High,ABOVE
Federated States of Micronesia,16,20,64,Federated_States_of_Micronesia,11.410921,High,ABOVE
Kiribati: Phoenix Islands,18,23,59,Kiribati:_Phoenix_Islands,15.900944,Moderate-High,ABOVE
Tuvalu,21,25,54,Tuvalu,20.624085,Moderate,ABOVE
Kiribati: Line Islands,25,32,43,Kiribati:_Line_Islands,14.624954,High,ABOVE
Austral Islands,26,32,42,Austral_Islands,11.493066,High,ABOVE
Tokelau,30,30,40,Tokelau,20.133254,Moderate,ABOVE
Pitcairn Islands,29,32,39,Pitcairn_Islands,13.405402,High,CLIMATOLOGY


### IMPORTANT, IF THE ATTRIBUTED CATEGORIES DO NOT MAKE SENSE, PLEASE CHANGE THEM 'MANUALLY'

if needed you can modify the category given for the forecast like this: 
        
```Python
forecast = forecast.loc['Marquesas','CATEGORY'] = 'AVG-BELOW'
```
----- 

### saves to the folder for inclusion in the PPT 

#### This is the table that appears on page 2 of the ICu bulletin

In [14]:
ppt_forecast_table = forecast.loc[:,['T1','T2','T3','CATEGORY','CONFIDENCE']]

In [15]:
ppt_forecast_table = ppt_forecast_table.sort_values(by=['T3','T2','T1'], ascending=False)

In [16]:
ppt_forecast_table = ppt_forecast_table.rename(index={'Tuamotu / Gambier Islands':'Tuamotu Islands'})

In [17]:
ppt_forecast_table = ppt_forecast_table.rename(index={'Federated States of Micronesia':'FSM'})

In [18]:
ppt_forecast_table.to_csv(dpath / 'ICU_forecast_table_for_PPT_{}.csv'.format(bulletin_dir))

### code the categories for the water stress calculations

In [19]:
dict_replace = {}

dict_replace['ABOVE'] = 4

dict_replace['AVG - ABOVE'] = 2
dict_replace['AVG-ABOVE'] = 2
dict_replace['AVG – ABOVE'] = 2
dict_replace['AVG–ABOVE'] = 2

dict_replace['NEAR NORMAL'] = 0
dict_replace['NEAR NORMAL '] = 0
dict_replace['CLIMATOLOGY'] = 0


dict_replace['AVG - BELOW'] = -3
dict_replace['AVG-BELOW'] = -3
dict_replace['AVG – BELOW'] = -3
dict_replace['AVG–BELOW'] = -3

dict_replace['BELOW'] = -5

dict_replace['CLIMATOLOGY'] = -666

In [20]:
forecast = forecast.replace({'CATEGORY':dict_replace})

In [21]:
forecast

,T1,T2,T3,country_fname,std,CONFIDENCE,CATEGORY
country_name,,,,,,,
Nauru,2,2,96,Nauru,26.095783,High,4
Kiribati: Gilbert Islands,3,4,93,Kiribati:_Gilbert_Islands,19.975656,High,4
Federated States of Micronesia,16,20,64,Federated_States_of_Micronesia,11.410921,High,4
Kiribati: Phoenix Islands,18,23,59,Kiribati:_Phoenix_Islands,15.900944,Moderate-High,4
Tuvalu,21,25,54,Tuvalu,20.624085,Moderate,4
Kiribati: Line Islands,25,32,43,Kiribati:_Line_Islands,14.624954,High,4
Austral Islands,26,32,42,Austral_Islands,11.493066,High,4
Tokelau,30,30,40,Tokelau,20.133254,Moderate,4
Pitcairn Islands,29,32,39,Pitcairn_Islands,13.405402,High,-666


### changes the names of the columns etc and saves the forecast table for mapping 

In [22]:
table_for_mapping = forecast.copy()

In [23]:
table_for_mapping.loc[:,'COUNTRYNAM'] = table_for_mapping.index

In [24]:
table_for_mapping.rename({'CATEGORY':'Stress'}, axis=1, inplace=True)

In [25]:
table_for_mapping = table_for_mapping.loc[:,['Stress','COUNTRYNAM']]

In [26]:
table_for_mapping.index.name = None

In [27]:
table_for_mapping.to_csv(dpath / 'ICU_forecast_table_for_mapping.csv')

### Now read the last 6 months rainfall categories from TRMM

In [29]:
last_6_months = pd.read_csv(dpath / 'last_6_months_TRMM_categories.csv', index_col=0)

In [30]:
last_6_months.set_index('month', inplace=True)

In [31]:
last_6_months

,Palau,Papua_New_Guinea,Society_Islands,Niue,Kiribati_Phoenix_Islands,Wallis_and_Futuna,Marquesas,Fiji,Kiribati_Line_Islands,Vanuatu_North,...,Tuvalu,Solomon_Islands,Tonga,Pitcairn_Islands,Guam,Northern_Marianas,Federated_States_of_Micronesia,Marshall_Islands,American_Samoa,year
month,,,,,,,,,,,,,,,,,,,,,
12,Normal,Above,Well below,Well above,Normal,Well above,Above,Well above,Normal,Well above,...,Normal,Well above,Well above,Below,Above,Below,Below,Above,Well above,2018
1,Below,Normal,Above,Above,Normal,Above,Normal,Well above,Above,Below,...,Below,Below,Well above,Well below,Normal,Below,Well above,Normal,Well below,2019
2,Well below,Well below,Normal,Below,Well above,Well above,Well below,Normal,Above,Above,...,Well above,Above,Well above,Above,Well above,Above,Below,Above,Well above,2019
3,Above,Well above,Normal,Well below,Above,Below,Below,Above,Above,Well below,...,Well below,Well below,Well above,Well above,Below,Well below,Above,Normal,Below,2019
4,Normal,Above,Above,Above,Well above,Well below,Below,Well above,Normal,Above,...,Above,Above,Normal,Below,Normal,Well below,Normal,Well below,Below,2019
5,Well below,Well above,Well below,Below,Normal,Well above,Below,Below,Well above,Well above,...,Well above,Below,Below,Below,Well below,Well above,Well below,Well above,Well above,2019


In [32]:
last_6_months = last_6_months.T

In [33]:
dict_replace = {}
dict_replace['Well below'] = -5 
dict_replace['Below'] = -3 
dict_replace['Normal'] = 0
dict_replace['Above'] = 2
dict_replace['Well above'] = 4

In [34]:
last_6_months.head()

month,12,1,2,3,4,5
Palau,Normal,Below,Well below,Above,Normal,Well below
Papua_New_Guinea,Above,Normal,Well below,Well above,Above,Well above
Society_Islands,Well below,Above,Normal,Normal,Above,Well below
Niue,Well above,Above,Below,Well below,Above,Below
Kiribati_Phoenix_Islands,Normal,Normal,Well above,Above,Well above,Normal


In [35]:
last_6_months  = last_6_months.replace(dict_replace)

In [36]:
last_6_months = last_6_months.sort_index()

In [37]:
last_6_months.columns = range(-6,0)

In [38]:
last_6_months = last_6_months.rename(index=dict_countries)

In [39]:
last_6_months.head()

,-6,-5,-4,-3,-2,-1
American Samoa,4,-5,4,-3,-3,4
Austral Islands,0,4,4,0,0,2
Federated States of Micronesia,-3,4,-3,2,0,-5
Fiji,4,4,0,2,4,-3
Guam,2,0,4,-3,0,-5


In [40]:
last_6_months.index.name = 'country_name'

### now calculates the water stress itself 

In [41]:
water_stress = forecast.loc[:,['CATEGORY']]

In [42]:
water_stress = water_stress.merge(last_6_months, left_index=True, right_index=True)

### replace the code for climatology (-666) with 0 (i.e. equivalent to normal) for calculation of the water stress level

In [43]:
water_stress = water_stress.replace({'CATEGORY':{-666:0}})

In [44]:
water_stress.loc[:,'Stress'] = water_stress.loc[:,-1] +  water_stress.loc[:,-2] + \
0.8 * water_stress.loc[:,-3] + \
0.6 * water_stress.loc[:,-4] + \
0.4 * water_stress.loc[:,-5] + \
0.2 * water_stress.loc[:,-6] + \
water_stress.loc[:,'CATEGORY']

In [45]:
water_stress.head()

,CATEGORY,-6,-5,-4,-3,-2,-1,Stress
country_name,,,,,,,,
Nauru,4,0,2,4,4,4,4,18.4
Kiribati: Gilbert Islands,4,2,4,4,4,4,4,19.6
Federated States of Micronesia,4,-3,4,-3,2,0,-5,-0.2
Kiribati: Phoenix Islands,4,0,0,4,2,4,0,12.0
Tuvalu,4,0,-3,4,-5,2,4,7.2


### now determine the water stress catergory from the water stress value

In [46]:
bins = [-np.inf, -10, -7, -2, 2, 6, 11, np.inf]

In [47]:
labels = [-3, -2, -1, 0, 1, 2, 3]

In [48]:
water_stress.loc[:,'stress_category'] = pd.cut(water_stress.Stress.values.flatten(), bins=bins, labels=labels)

In [49]:
water_stress

,CATEGORY,-6,-5,-4,-3,-2,-1,Stress,stress_category
country_name,,,,,,,,,
Nauru,4,0,2,4,4,4,4,18.4,3
Kiribati: Gilbert Islands,4,2,4,4,4,4,4,19.6,3
Federated States of Micronesia,4,-3,4,-3,2,0,-5,-0.2,0
Kiribati: Phoenix Islands,4,0,0,4,2,4,0,12.0,3
Tuvalu,4,0,-3,4,-5,2,4,7.2,2
Kiribati: Line Islands,4,0,2,2,2,0,4,11.6,3
Austral Islands,4,0,4,4,0,0,2,10.0,2
Tokelau,4,-3,0,2,-5,-5,4,-0.4,0
Pitcairn Islands,0,-3,-5,2,4,-3,-3,-4.2,-1


In [50]:
water_stress.index.name = 'locationID'

In [51]:
dpath

PosixPath('/home/nicolasf/operational/ICU/ops/bulletin/JJA_2019')

### now saves for mapping 

In [52]:
water_stress.to_csv(dpath / 'ICU_stress_table_for_mapping.csv')